In [2]:
import pandas as pd
import pyspark.sql.dataframe
from pca import pca
from pyspark.sql.functions import max, countDistinct, when, lit, pandas_udf, PandasUDFType, monotonically_increasing_id, split
from pyspark.sql.functions import pandas_udf, PandasUDFType, lit, col, collect_set, explode, countDistinct, when, monotonically_increasing_id, sum as spark_sum
from typing import List, Dict
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField, BooleanType
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import ClusterCentroids
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
# from src.exceptions.rca_base_exception import RCABaseException

In [3]:
import sys
print(sys.executable)

D:\developer\anaconda3\envs\MySpark\python.exe


In [4]:
import os
import json
import warnings
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession

os.environ['PYSPARK_PYTHON'] = '/usr/local/python-3.9.13/bin/python3'
warnings.filterwarnings('ignore')

os.environ['PYSPARK_PYTHON'] = '/usr/local/python-3.9.13/bin/python3'
spark = SparkSession.builder \
    .appName("pandas_udf") \
    .config('spark.sql.session.timeZone', 'Asia/Shanghai') \
    .config("spark.scheduler.mode", "FAIR") \
    .config('spark.driver.memory', '8g') \
    .config('spark.driver.cores', '12') \
    .config('spark.executor.memory', '8g') \
    .config('spark.executor.cores', '12') \
    .config('spark.cores.max', '12') \
    .config('spark.driver.host', '192.168.22.28') \
    .master("spark://192.168.12.47:7077,192.168.12.48:7077") \
    .getOrCreate()

[scatterd] >WARNING> 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


In [6]:
df_pandas = pd.read_csv(
    "D:/Jupyterfiles/晶合MVAFDC_general开发/MVAanlysisDevelop/uva_algorithm/uva调试1/train_data_pandas1.csv")

df_spark = ps.from_pandas(df_pandas).to_spark()
print(f"df_spark shape: ({df_spark.count()}, {len(df_spark.columns)})")
df_spark.show()

df_spark shape: (345562, 12)
+----------+---------+--------+----------+------+--------+--------------+--------+------------+--------------------+----------------+-----+
|Unnamed: 0| WAFER_ID|  OPE_NO|CHAMBER_ID|RUN_ID|EQP_NAME|    PRODUCT_ID|  PRODG1|CHAMBER_NAME|     PARAMETRIC_NAME|STATISTIC_RESULT|label|
+----------+---------+--------+----------+------+--------+--------------+--------+------------+--------------------+----------------+-----+
|         0|NGF394-12|1F.EEK10|     11342|159816|   EKT72|AFGN8S01N.0C01|L15HL02A|   EKT72_PM2|FLOWSPLITCENTER#A...|            4.33|    0|
|         1|NGF394-12|1F.EEK10|     11342|159816|   EKT72|AFGN8S01N.0C01|L15HL02A|   EKT72_PM2|EDGE_HE_FLOW#AOTU...|           2.045|    0|
|         2|NGF394-12|1F.EEK10|     11342|159816|   EKT72|AFGN8S01N.0C01|L15HL02A|   EKT72_PM2|LOWERMAGNETGAPPOS...|             0.0|    0|
|         3|NGF394-12|1F.EEK10|     11342|159816|   EKT72|AFGN8S01N.0C01|L15HL02A|   EKT72_PM2|ESC_CURRENT#AOTU_...|            60.

In [67]:
oper, prod, eqp = '1F.EEK10', 'AFKN4X01N.0B01', 'EKT72'
df11 = df_pandas.query(f"OPE_NO == '{oper}' and PRODUCT_ID == '{prod}' and EQP_NAME == '{eqp}'")
df11[df11['PARAMETRIC_NAME'] == 'LO_RF_VPP#AOTU_STEP_2#MEAN']['label'].value_counts()

1    269
0    167
2     38
Name: label, dtype: int64

In [11]:
grpby_list = ['OPE_NO', 'PRODUCT_ID', 'EQP_NAME']

In [9]:
oper, prod, eqp = '1F.EEK10', 'AFGN1S01N.0B02', 'EKT72'
df = df_pandas.query(f"OPE_NO == '{oper}' and PRODUCT_ID == '{prod}' and EQP_NAME == '{eqp}'")

In [12]:
index_cols = ['WAFER_ID', 'label']
columns_cols = grpby_list + ['PARAMETRIC_NAME']
df_pivot = df.dropna(axis=0).pivot_table(index=index_cols,
                                         columns=columns_cols,
                                         values=['STATISTIC_RESULT'])
df_pivot.columns = df_pivot.columns.map('#'.join)
df_pivot = df_pivot.fillna(df_pivot.mean()).reset_index(drop=False)
# Remove completely identical columns
for column in df_pivot.columns.difference(index_cols):
    if df_pivot[column].nunique() == 1:
        df_pivot = df_pivot.drop(column, axis=1)

In [14]:
df_pivot['label'].value_counts()

1    298
2    178
0     49
Name: label, dtype: int64

In [50]:
df_pivot

,WAFER_ID,label,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#APC_POSITION#AOTU_STEP_4#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#APC_POSITION#AOTU_STEP_4#RANGE,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#APC_POSITION#AOTU_STEP_6#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#APC_POSITION#AOTU_STEP_6#RANGE,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#APC_POSITION#AOTU_STEP_7#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#APC_POSITION#AOTU_STEP_7#RANGE,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#BOTTOMFLOWRATE#AOTU_STEP_4#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#BOTTOMFLOWRATE#AOTU_STEP_6#MEAN,...,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#PROCESS_GAS_6_CF4#AOTU_STEP_4#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#PROCESS_GAS_6_CF4#AOTU_STEP_6#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#PROCESS_GAS_6_CF4#AOTU_STEP_7#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#PROCESS_GAS_6_CF4#WINDOW_1#SUM,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#UPPER_TEMPERATURE#AOTU_STEP_4#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#UPPER_TEMPERATURE#AOTU_STEP_6#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#UPPER_TEMPERATURE#AOTU_STEP_7#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#WALL_TEMPERATURE#AOTU_STEP_4#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#WALL_TEMPERATURE#AOTU_STEP_6#MEAN,STATISTIC_RESULT#1F.EEK10#AFGN1S01N.0B02#EKT72#WALL_TEMPERATURE#AOTU_STEP_7#MEAN
0,NGE708-01,1,5.4429,0.1,6.3023,0.1,6.3000,0.0,30.0000,30.0046,...,239.9000,171.9917,169.9019,345467.4,59.4952,60.9333,60.2476,61.5429,60.2253,61.1143
1,NGE708-02,2,5.4238,0.1,6.3023,0.2,6.3016,0.1,29.9952,29.9942,...,239.9000,171.9993,169.9219,345440.9,58.4429,59.7837,61.2651,61.7190,60.2698,60.6698
2,NGE708-03,2,5.4238,0.1,6.3035,0.2,6.3032,0.1,30.0000,30.0000,...,239.9000,171.9999,169.9181,343646.2,58.3476,58.7744,60.5619,61.2429,60.4163,60.0730
3,NGE708-04,1,5.4238,0.1,6.3046,0.2,6.3000,0.0,30.0000,29.9966,...,239.9014,171.9994,169.9013,343131.0,58.8429,58.5011,59.9887,59.3429,60.8506,60.3435
4,NGE708-05,1,5.4190,0.1,6.3034,0.2,6.3000,0.0,30.0000,30.0034,...,239.9000,171.9990,169.9023,344208.9,61.2333,59.3241,58.8143,59.2619,60.9437,60.4286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,NGG012-21,1,5.4286,0.2,6.3034,0.2,6.3000,0.0,30.0143,30.0000,...,239.9000,170.9015,169.9048,340284.3,59.5952,58.2069,58.7032,59.6714,60.9092,60.1226
521,NGG012-22,2,5.4238,0.1,6.3023,0.2,6.3000,0.0,29.9857,30.0000,...,239.9000,170.9022,169.9012,340297.0,61.1000,60.9216,58.7267,59.2667,60.9386,60.5633
522,NGG012-23,1,5.4238,0.2,6.3034,0.2,6.3000,0.0,30.0000,29.9954,...,239.9000,170.9016,169.9102,340942.3,58.5667,60.0345,61.4098,59.6333,60.7379,60.9836
523,NGG012-24,2,5.4333,0.1,6.3011,0.1,6.3000,0.0,30.0000,30.0057,...,239.9000,170.9023,169.9062,340819.0,58.5286,57.9632,59.0836,60.4095,60.4253,61.3180


In [26]:
x_train = df_pivot[df_pivot.columns.difference(['WAFER_ID', 'label']).tolist()]
y_train = df_pivot[['label']]

In [27]:
pipe, param_grid = FitModelForUvaData.get_pipe_params(model='rf')

In [55]:
grid = GridSearchCV(estimator=pipe, scoring='balanced_accuracy', param_grid=param_grid,  n_jobs=-1)
grid.fit(x_train.values, y_train.values.ravel())

GridSearchCV(estimator=Pipeline(steps=[('imputer',
                                        SimpleImputer(fill_value=-999,
                                                      strategy='constant')),
                                       ('scaler', StandardScaler()),
                                       ('model',
                                        RandomForestClassifier(random_state=2000))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [5, 15, 25, 35, 45],
                         'model__min_samples_leaf': [1, 3],
                         'model__min_samples_split': [2, 5],
                         'model__n_estimators': [10, 20, 30, 40, 50]},
             scoring='balanced_accuracy')

In [56]:
best_est = grid.best_estimator_.steps[-1][-1]
best_est

RandomForestClassifier(max_depth=25, n_estimators=40, random_state=2000)

In [57]:
grid.best_estimator_

Pipeline(steps=[('imputer',
                 SimpleImputer(fill_value=-999, strategy='constant')),
                ('scaler', StandardScaler()),
                ('model',
                 RandomForestClassifier(max_depth=25, n_estimators=40,
                                        random_state=2000))])

In [58]:
grid.best_score_

0.4340857322213254

In [25]:
class FitModelForUvaData:
    @staticmethod
    def get_pivot_table(df: pd.DataFrame, grpby_list: List[str]) -> pd.DataFrame:
        """
        Pivot the DataFrame based on specified grouping columns.

        Parameters:
        - df: Data for modeling.
        - grpby_list: List of grouping columns.

        Returns:
        - DataFrame: Result of pivoting the table.
        """
        index_cols = ['WAFER_ID', 'label']
        columns_cols = grpby_list + ['PARAMETRIC_NAME']
        df_pivot = df.dropna(axis=0).pivot_table(index=index_cols,
                                                 columns=columns_cols,
                                                 values=['STATISTIC_RESULT'])
        df_pivot.columns = df_pivot.columns.map('#'.join)
        df_pivot = df_pivot.fillna(df_pivot.mean()).reset_index(drop=False)
        # Remove completely identical columns
        for column in df_pivot.columns.difference(index_cols):
            if df_pivot[column].nunique() == 1:
                df_pivot = df_pivot.drop(column, axis=1)
        return df_pivot

    @staticmethod
    def get_pipe_params(model):
        common_steps = [
            ('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
            ('scaler', StandardScaler())
        ]
        models = {
            'rf': (RandomForestClassifier(random_state=2000), {
                'model__n_estimators': [*range(10, 60, 10)],
                'model__max_depth': [*range(5, 50, 10)],
                'model__min_samples_split': [2, 5],
                'model__min_samples_leaf': [1, 3]
            }),

            'decisionTree': (DecisionTreeClassifier(random_state=2024), {
                'model__max_depth': [None, 5, 10, 15],
                'model__min_samples_split': [2, 5, 10],
                'model__min_samples_leaf': [1, 2, 4]
            }),

            'svc': (LinearSVC(random_state=2024, fit_intercept=False), {
                'model__loss': ['hinge', 'squared_hinge'],
                'model__C': [0.1, 0.5, 1, 10, 50]
            }),

            'logistic': (LogisticRegression(random_state=2024, fit_intercept=False, solver='liblinear'), {
                'model__penalty': ['l1', 'l2'],
                'model__C': [0.1, 0.5, 1, 10, 50]
            }),

            'sgd': (SGDClassifier(random_state=2024, fit_intercept=False), {
                'model__loss': ['hinge', 'log_loss', 'perceptron', 'huber'],
                'model__penalty': ['l1', 'l2', 'elasticnet', None],
                'model__alpha': [0.0001, 0.001, 0.01, 0.1],
                'model__max_iter': [100, 500, 1000]
            })
        }

        if model in models:
            model_class, param_grid = models[model]
            steps = common_steps + [('model', model_class)]
            pipe = Pipeline(steps)
            return pipe, param_grid
        else:
            return None, None

    @staticmethod
    def construct_features_when_not_satisfied(df_run, df_pivot, x_train, grpby_list, model_condition):
        x_len = len(x_train.columns)

        if model_condition == 'classification':

            small_importance_res = pd.DataFrame({"features": x_train.columns,
                                                 "importance": [0.0] * x_len})

            sample_res_dict = {'roc_auc_score': 0.0,
                               'algorithm_satisfied': 'FALSE',
                               'x_train_shape': str(x_train.shape)}
            sample_res_dict.update({col_: df_run[col_].values[0] for col_ in grpby_list})
            small_sample_res = pd.DataFrame(sample_res_dict, index=[0])
            res_top_select = pd.concat([small_importance_res, small_sample_res])
            return res_top_select

        elif model_condition == 'pca':

            res_top_select = pd.DataFrame({"features": x_train.columns,
                                           "importance": [0.0] * x_len,
                                           "algorithm_satisfied": ['FALSE'] * x_len,
                                           "x_train_shape": [str(x_train.shape)] * x_len})
            for col_ in grpby_list:
                res_top_select[col_] = df_run[col_].values[0]
            return res_top_select

    @staticmethod
    def fit_classification_model(df: pyspark.sql.DataFrame, grpby_list: List[str], model) -> pyspark.sql.DataFrame:
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("roc_auc_score", FloatType(), True),
                              StructField("features", StringType(), True),
                              StructField("importance", FloatType(), True),
                              StructField("algorithm_satisfied", StringType(), True),
                              StructField("x_train_shape", StringType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run: pd.DataFrame) -> pd.DataFrame:
            # Pivot the table
            df_pivot = FitModelForUvaData.get_pivot_table(df=df_run, grpby_list=grpby_list)

            # Define independent and dependent variables
            x_train = df_pivot[df_pivot.columns.difference(['WAFER_ID', 'label']).tolist()]
            y_train = df_pivot[['label']]

            if x_train.shape[1] > 1 and y_train['label'].nunique() > 1:
                pipe, param_grid = FitModelForUvaData.get_pipe_params(model=model)
                try:
                    grid = GridSearchCV(estimator=pipe, scoring='roc_auc', param_grid=param_grid, cv=3, n_jobs=-1)
                    grid.fit(x_train.values, y_train.values.ravel())
                except ValueError:
                    return FitModelForUvaData.construct_features_when_not_satisfied(df_run, df_pivot, x_train,
                                                                                    grpby_list, 'classification')

                best_est = grid.best_estimator_.steps[-1][-1]
                if hasattr(best_est, 'feature_importances_'):
                    small_importance_res = pd.DataFrame(
                        {'features': x_train.columns, 'importance': best_est.feature_importances_})
                else:
                    small_importance_res = pd.DataFrame(
                        {'features': x_train.columns, 'importance': abs(best_est.coef_.ravel())})

                sample_res_dict = {'roc_auc_score': 0.0 if np.isnan(grid.best_score_) else grid.best_score_,
                                   'algorithm_satisfied': 'TRUE',
                                   'x_train_shape': str(x_train.shape)}
                sample_res_dict.update({col_: df_run[col_].values[0] for col_ in grpby_list})
                small_sample_res = pd.DataFrame(sample_res_dict, index=[0])
                res_top_select = pd.concat([small_importance_res, small_sample_res])
                return res_top_select
            else:
                res_top_select = FitModelForUvaData.construct_features_when_not_satisfied(df_run, df_pivot, x_train,
                                                                                          grpby_list, 'classification')
                return res_top_select

        return df.groupby(grpby_list).apply(get_model_result)

    @staticmethod
    def construct_features_when_satisfy_pca(df_run, df_pivot, x_train, grpby_list) -> pd.DataFrame:
        # 得到PCA算法结果res_top_select
        n_components = min(min(x_train.shape) - 2, 20)
        model = pca(n_components=n_components, verbose=None, random_state=2024)
        results = model.fit_transform(x_train)
        res_top = results['topfeat']
        res_top_select = res_top[res_top['type'] == 'best'][['feature', 'loading']]
        res_top_select['importance'] = abs(res_top_select['loading'])
        res_top_select = res_top_select.rename(columns={'feature': 'features'}).drop("loading",
                                                                                     axis=1).drop_duplicates()
        for col_ in grpby_list:
            res_top_select[col_] = df_run[col_].values[0]
        res_top_select['x_train_shape'] = str(x_train.shape)
        res_top_select['algorithm_satisfied'] = 'TRUE'
        return res_top_select

    @staticmethod
    def fit_pca_model(df: pyspark.sql.dataframe, grpby_list: List[str]) -> pyspark.sql.dataframe:
        # Dynamically build schema according to the grpby_list
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("features", StringType(), True),
                              StructField("importance", FloatType(), True),
                              StructField("algorithm_satisfied", StringType(), True),
                              StructField("x_train_shape", StringType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run: pd.DataFrame) -> pd.DataFrame:
            df_pivot = FitModelForUvaData.get_pivot_table(df=df_run, grpby_list=grpby_list)
            df_pivot_copy = df_pivot.copy()
            df_pivot_all = pd.concat([df_pivot, df_pivot_copy], axis=0)

            x_train = df_pivot_all[df_pivot_all.columns.difference(['WAFER_ID', 'label']).tolist()]

            if min(x_train.shape) > 2:
                res_top_select = FitModelForUvaData.construct_features_when_satisfy_pca(df_run, df_pivot, x_train,
                                                                                        grpby_list)
                return res_top_select
            else:
                res_top_select = FitModelForUvaData.construct_features_when_not_satisfied(df_run, df_pivot, x_train,
                                                                                          grpby_list, 'pca')
                return res_top_select

        return df.groupby(grpby_list).apply(get_model_result)